# Obtain Hex Addresses

We will use reverse geocoding to find the nearest address to each hex centre.

In [1]:
import geopandas as gpd
import numpy as np
import plotly.express as px
from diskcache import Cache
from geopy import Nominatim
from geopy.distance import distance
from ratelimiter import RateLimiter
from tqdm.notebook import tqdm

In [2]:
df_hex = gpd.read_file('../data/BBMP_hex.geojson')
print(df_hex.shape)
df_hex.head()

(942, 9)


,id,hex_id,ward_no,centre_lat,centre_lon,resolution,pop_total,ward_name,geometry
0,8860169669fffff,8860169669fffff,1,13.098170,77.600839,8,1186.041718,Kempegowda Ward,"POLYGON ((77.60501 13.09567, 77.60510 13.10065..."
1,8860169661fffff,8860169661fffff,1,13.105662,77.596750,8,1845.480809,Kempegowda Ward,"POLYGON ((77.60092 13.10316, 77.60101 13.10814..."
2,8860169753fffff,8860169753fffff,1,13.128114,77.592914,8,1786.194897,Kempegowda Ward,"POLYGON ((77.59709 13.12561, 77.59717 13.13060..."
3,8861892db3fffff,8861892db3fffff,1,13.113109,77.609520,8,1413.219880,Kempegowda Ward,"POLYGON ((77.61369 13.11060, 77.61378 13.11559..."
4,8860169645fffff,8860169645fffff,1,13.090701,77.596498,8,2635.116823,Kempegowda Ward,"POLYGON ((77.60067 13.08820, 77.60076 13.09318..."


## Reverse Geocoding

Data is loaded from the GeoJSON file created earlier and displayed.

We obtain the closest addresses of each hex centre using Nominatim reverse geocoding. We need to cache the responses received, since querying Nominatim is quite slow, and we want to avoid repeating requests.

In [3]:
# Set cache location for Nominatim requests
nom_cache = '../data/cache/nominatim'

# Create Nominatim geocoder object with custom user agent as required by their terms of service
geocoder = Nominatim(user_agent = 'coursera_capstone')

# Create RateLimiter object to ensure we don't exceed Nominatim's 1 reqest per second rule
limiter = RateLimiter(max_calls = 1, period = 1)

In [4]:
# Empty lists to store responses
address = []
addr_coord = []

points = df_hex[['centre_lat', 'centre_lon']].to_records()

queries = 0 # Added variable to track number of records retrieved from cache

with Cache(nom_cache) as cache:
    for p in tqdm(points):
        query = (p.centre_lat, p.centre_lon)
        key = str(query) #! key must be a unique string
        
        if key in cache:
            response = cache[key] # Read cached value
        else:
            with limiter:
                response = geocoder.reverse(query, timeout = 30, addressdetails=True)
                cache[key] = response # Set cache value
                queries += 1
            
        address.append(response.address)
        addr_coord.append((response.latitude, response.longitude))
print('{} new queries made.'.format(queries))

  0%|          | 0/942 [00:00<?, ?it/s]

0 new queries made.


## Calculating Distances

We use geopy to calculate the geodesic distance between each hexagon centre and the address obtained. We want to check if the addresses provided are close to the hex centres.

In [5]:
centre_coord = df_hex[['centre_lat', 'centre_lon']].to_records(index=False)

address_error = [] # Empty list to store data

for centre, addr in zip(centre_coord, addr_coord):
    dist = distance(centre, addr).meters
    address_error.append(dist) # Use Geopy function

# Plot histogram
address_err_fig = px.histogram(
    x = address_error,
    histnorm = 'percent',
    #cumulative = True,
    template = 'plotly',
)

address_err_fig.update_layout(
    title = "Error in address locations",
    title_x = 0.5,
    xaxis_title = 'Distance between Hex centre & Address (meters)',
    yaxis_title = 'Percentage of hexes',
    bargap = 0.01,
)

The reverse geocoding is accurate enough. Almost all of the addresses returned are within a few hundred meters of the hexagon centre. In any case, we will mainly be using the hex centres for future functions - the address is mainly for reference.

In [6]:
# Assign to new columns
df_hex['address'] = address
df_hex.to_feather('../data/bangalore_hex_addresses.feather') # Save file
df_hex.head() # Display final table

<ipython-input-6-61f42f6f3292>:3: UserWarning:

this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')



,id,hex_id,ward_no,centre_lat,centre_lon,resolution,pop_total,ward_name,geometry,address
0,8860169669fffff,8860169669fffff,1,13.098170,77.600839,8,1186.041718,Kempegowda Ward,"POLYGON ((77.60501 13.09567, 77.60510 13.10065...","Yelahanka Sante, Shivanahalli Main Road, Gandh..."
1,8860169661fffff,8860169661fffff,1,13.105662,77.596750,8,1845.480809,Kempegowda Ward,"POLYGON ((77.60092 13.10316, 77.60101 13.10814...","Yelahanka, Kempegowda, Yelahanka Zone, Bengalu..."
2,8860169753fffff,8860169753fffff,1,13.128114,77.592914,8,1786.194897,Kempegowda Ward,"POLYGON ((77.59709 13.12561, 77.59717 13.13060...","Century Artizan, Kempegowda, Yelahanka Zone, B..."
3,8861892db3fffff,8861892db3fffff,1,13.113109,77.609520,8,1413.219880,Kempegowda Ward,"POLYGON ((77.61369 13.11060, 77.61378 13.11559...","Yelahanka, Kempegowda, Yelahanka Zone, Bengalu..."
4,8860169645fffff,8860169645fffff,1,13.090701,77.596498,8,2635.116823,Kempegowda Ward,"POLYGON ((77.60067 13.08820, 77.60076 13.09318...","Bellary Road, Amruthnagar, Byatarayanapura, Ye..."
